In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
torch.cuda.empty_cache()

from typing import Dict, Tuple
from datasets import list_datasets, load_dataset, DatasetDict,Dataset
from collections import Counter
from typing import List, Dict, Union, Callable, Any
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pprint import pprint
import torch.nn as nn
import evaluate
from transformers import AutoTokenizer, T5ForConditionalGeneration, AutoConfig
from evaluate import TextClassificationEvaluator, Metric, EvaluationModuleInfo

In [ ]:
import transformers

print(transformers.__version__)

4.26.1


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device) 

cuda


## Preparing the dataset

In [ ]:
# dataset: DatasetDict = load_dataset("Sree1994/babylm_childstories")

ds_train = load_dataset("Sree1994/babylm_100M", split="train")
ds_valid = load_dataset("Sree1994/babylm_100M", split="valid")


raw_datasets = DatasetDict(
    {
        "train": ds_train,
        "valid": ds_valid
    }
)

raw_datasets

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/35000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/255000 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/35000 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/Sree1994___parquet/Sree1994--babylm_100M-d2dafd507276a5eb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 255000
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 35000
    })
})

In [ ]:
model_checkpoint = "t5-small"

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, Trainer, TrainingArguments,AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
context_length = 128
vocab_size = tokenizer.vocab_size

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
config = AutoConfig.from_pretrained(
    "t5-small",
    vocab_size=len(tokenizer),
    random_init=True
)
print(len(tokenizer))
model = T5ForConditionalGeneration(config)
model.init_weights()
model_size = sum(t.numel() for t in model.parameters())
print(f"T5 size: {model_size/1000**2:.1f}M parameters")

32100
T5 size: 60.5M parameters


In [ ]:
def preprocess_function(examples):
    inputs = examples['text']
    targets = examples['text']
    model_inputs = tokenizer(inputs, padding='max_length', truncation=True, max_length=128)
    with tokenizer.as_target_tokenizer():
        model_targets = tokenizer(targets, padding='max_length', truncation=True, max_length=128)
    model_inputs['labels'] = model_targets['input_ids']
    return model_inputs

# Preprocess the data
dataset = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/255000 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/35000 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='./100M/results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 255000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 47814
  Number of trainable parameters = 60492288


Epoch,Training Loss,Validation Loss
1,1.814500,1.459210
2,0.063100,0.003558
3,0.009600,0.000341


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 35000
  Batch size = 16
Saving model checkpoint to ./100M/results/checkpoint-15938
Configuration saved in ./100M/results/checkpoint-15938/config.json
Configuration saved in ./100M/results/checkpoint-15938/generation_config.json
Model weights saved in ./100M/results/checkpoint-15938/pytorch_model.bin
tokenizer config file saved in ./100M/results/checkpoint-15938/tokenizer_config.json
Special tokens file saved in ./100M/results/checkpoint-15938/special_tokens_map.json
Copy vocab file to ./100M/results/checkpoint-15938/spiece.model
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If text are not 

TrainOutput(global_step=47814, training_loss=0.9831894430731069, metrics={'train_runtime': 17971.9302, 'train_samples_per_second': 42.566, 'train_steps_per_second': 2.66, 'total_flos': 2.588411953152e+16, 'train_loss': 0.9831894430731069, 'epoch': 3.0})

In [ ]:
model = trainer.model  # make sure to load_best_model_at_end=True!

# run a final evaluation on the test set
val = trainer.evaluate(metric_key_prefix="test", eval_dataset=dataset['valid'])

The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: text. If text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 35000
  Batch size = 16


In [ ]:
trn.training_loss

In [ ]:
valid = val.get("test_loss")
torch.exp(torch.tensor(valid))

tensor(1.0003)